Evaluating the model trained with **Sentinel-2 L2A** data (natural-looking images).

# 0. Setup

## 0.1. Libraries

In [ ]:
# Utilities
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report

# Deep learning
import torch
from torch import nn, utils
from torchmetrics import classification

# Custom library
from library import nn_model, utilities, visualizations, preprocessing

# Device
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
device = torch.device(device)
print(f'Using device: {device}')

# Utilities
# classes = {0:'non-building', 255:'building'}
seed = 42
utilities.set_seed(seed)

Using device: cuda
Seed set to 42 for NumPy, Torch and Random for reproducibility.


## 0.2. Setting the path to the data

We set the path to the tiles that have been created in the preprocessing notebook, `3_preprocessing.ipynb`.

In [ ]:
base_input_dir = '/media/pablo/Shared files/data/'  # Adjust this path to your data directory containing the labelled dataset
input_labelled_dir = os.path.join(base_input_dir, 'Satellite_burned_area_dataset')  # Path to the original labelled dataset
tile_dir = os.path.join(base_input_dir, 'tiled_labelled_dataset')  # Path to the tiled dataset
sentinel_type = 2  # Sentinel-2 data

## 0.3. Notebook description

In this notebook we evaluate the model trained on Sentinel-2 L2A images on the test split made in that notebook.

> For consistency, in this notebook we again create the same split in the same way, but in this case we will only be using the test data. **Please ensure that the seeds set in both notebooks are the same for avoiding data leakage and creating the same test set in both notebooks**. Furthermore, pre-processing must be implemented in the same way as for the training notebook with Sentinel-2 L2A images.